# Imports

In [1]:
config = {
    'dhdt': {
        'depth': 3,
        'learning_rate': 0.01,#1e-3,
        
        'initializer': 'he_normal', #GlorotUniform
        
        'loss': 'binary_crossentropy',#'mae',
        'optimizer': 'adam',        
        
        'beta_1': 10,
        'beta_2': 50,
        
        'squeeze_factor': 1,
        
        'batch_size': 512,
        'epochs': 1_000,
        'early_stopping_epochs': 50,
    },
    
    
    
    'make_classification': {
        'number_of_variables': 20,
        'n_samples': 5_000,
        'num_eval': 50,
    },

    'computation': {
        'random_seed': 42,
        'trials': 5,
        'n_jobs': 50,
        'verbosity': 0,
    },
}



In [2]:
import numpy as np

import sklearn
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder, OrdinalEncoder

from livelossplot import PlotLosses

import os
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt

from IPython.display import Image
from IPython.display import display, clear_output

import pandas as pd

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = '' #'true'

import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import logging

import tensorflow as tf
import tensorflow_addons as tfa

tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(3)

np.seterr(all="ignore")

from keras import backend as K
from keras.utils.generic_utils import get_custom_objects


import seaborn as sns
sns.set_style("darkgrid")

import time
import random

from utilities.utilities import *
from utilities.DHDT import *

from joblib import Parallel, delayed

from itertools import product
from collections.abc import Iterable

from copy import deepcopy

# Evaluation

## make_classification

In [3]:
if True:
    metrics = ['accuracy', 'f1']
    
    config_test = deepcopy(config)
    config_test['make_classification']['n_samples'] = 10_000
    config_test['dhdt']['epochs'] = 100

    dataset_dict = {}
    model_dict = {}

    scores_dict = {'sklearn': {},
                   'DHDT': {}}

    dataset_dict = get_preprocessed_dataset('make_classification',
                                            random_seed=config_test['computation']['random_seed'],
                                            config=config_test['make_classification'],
                                            verbosity=1)

    model_dict['sklearn'] = DecisionTreeClassifier(max_depth=3, 
                                                   random_state=config_test['computation']['random_seed'])

    model_dict['sklearn'].fit(dataset_dict['X_train'], 
                              dataset_dict['y_train'])



    model_dict['DHDT'] = DHDT(dataset_dict['X_train'].shape[1],

                                depth = config_test['dhdt']['depth'],

                                learning_rate = config_test['dhdt']['learning_rate'],
                                optimizer = config_test['dhdt']['optimizer'],

                                beta_1 = config_test['dhdt']['beta_1'],
                                beta_2 = config_test['dhdt']['beta_2'],

                                squeeze_factor = config_test['dhdt']['squeeze_factor'],

                                loss = config_test['dhdt']['loss'],#'mae',

                                random_seed = config_test['computation']['random_seed'],
                                verbosity = 2)        


    scores_dict['history'] = model_dict['DHDT'].fit(dataset_dict['X_train'], 
                                                  dataset_dict['y_train'], 
                                                  batch_size=config_test['dhdt']['batch_size'], 
                                                  epochs=config_test['dhdt']['epochs'], 
                                                  early_stopping_epochs=config_test['dhdt']['early_stopping_epochs'], 
                                                  valid_data=(dataset_dict['X_valid'], dataset_dict['y_valid']))



    dataset_dict['y_test_dhdt'] = model_dict['DHDT'].predict(dataset_dict['X_test'])
    dataset_dict['y_valid_dhdt'] = model_dict['DHDT'].predict(dataset_dict['X_valid'])

    dataset_dict['y_test_sklearn'] = model_dict['sklearn'].predict(dataset_dict['X_test'])
    dataset_dict['y_valid_sklearn'] = model_dict['sklearn'].predict(dataset_dict['X_valid'])     
    
    for metric in metrics:
        
        if metric in ['accuracy', 'f1']:
            y_test_dhdt = np.round(dataset_dict['y_test_dhdt'])
            y_valid_dhdt = np.round(dataset_dict['y_valid_dhdt'])
            y_test_sklearn = np.round(dataset_dict['y_test_sklearn'])
            y_valid_sklearn = np.round(dataset_dict['y_valid_sklearn'])         
        else:
            y_test_dhdt = dataset_dict['y_test_dhdt']
            y_valid_dhdt = dataset_dict['y_valid_dhdt']
            y_test_sklearn = dataset_dict['y_test_sklearn']
            y_valid_sklearn =    dataset_dict['y_valid_sklearn']                
        
        scores_dict['sklearn'][metric + '_test'] = sklearn.metrics.get_scorer(metric)._score_func(dataset_dict['y_test'], y_test_sklearn)
        scores_dict['DHDT'][metric + '_test'] = sklearn.metrics.get_scorer(metric)._score_func(dataset_dict['y_test'], y_test_dhdt)

        scores_dict['sklearn'][metric + '_valid'] = sklearn.metrics.get_scorer(metric)._score_func(dataset_dict['y_valid'], y_valid_sklearn)   
        scores_dict['DHDT'][metric + '_valid'] = sklearn.metrics.get_scorer(metric)._score_func(dataset_dict['y_valid'], y_valid_dhdt)


Original Data Shape (selected):  (10000, 20)
Original Data Shape (encoded):  (10000, 20)
Original Data Class Distribution:  4990  (true) / 5010  (false)
(7000, 20) (7000,)
(1000, 20) (1000,)
(2000, 20) (2000,)
True Ratio:  0.5008571428571429


epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 00 | Loss: 0.69326 |
Epoch: 01 | Loss: 0.69328 |
Epoch: 02 | Loss: 0.69365 |
Epoch: 03 | Loss: 0.69297 |
Epoch: 04 | Loss: 0.69146 |
Epoch: 05 | Loss: 0.68948 |
Epoch: 06 | Loss: 0.68578 |
Epoch: 07 | Loss: 0.68183 |
Epoch: 08 | Loss: 0.67854 |
Epoch: 09 | Loss: 0.67689 |
Epoch: 10 | Loss: 0.67602 |
Epoch: 11 | Loss: 0.67546 |
Epoch: 12 | Loss: 0.67463 |
Epoch: 13 | Loss: 0.67363 |
Epoch: 14 | Loss: 0.67328 |
Epoch: 15 | Loss: 0.67334 |
Epoch: 16 | Loss: 0.67288 |
Epoch: 17 | Loss: 0.67408 |
Epoch: 18 | Loss: 0.67336 |
Epoch: 19 | Loss: 0.67302 |
Epoch: 20 | Loss: 0.67306 |
Epoch: 21 | Loss: 0.67438 |
Epoch: 22 | Loss: 0.67353 |
Epoch: 23 | Loss: 0.67322 |
Epoch: 24 | Loss: 0.67312 |
Epoch: 25 | Loss: 0.67332 |
Epoch: 26 | Loss: 0.67368 |
Epoch: 27 | Loss: 0.67309 |
Epoch: 28 | Loss: 0.67356 |
Epoch: 29 | Loss: 0.67317 |
Epoch: 30 | Loss: 0.67363 |
Epoch: 31 | Loss: 0.67322 |
Epoch: 32 | Loss: 0.67368 |
Epoch: 33 | Loss: 0.67319 |
Epoch: 34 | Loss: 0.67321 |
Epoch: 35 | Loss: 0.

In [4]:
if True:
    parallel_eval_synthetic = Parallel(n_jobs=config['computation']['n_jobs'], verbose=3, backend='loky') #loky #sequential multiprocessing
    evaluation_results_synthetic = parallel_eval_synthetic(delayed(evaluate_synthetic_parallel)(index = index,
                                                                                                random_seed_data = config['computation']['random_seed']+index,
                                                                                                random_seed_model = config['computation']['random_seed'],#+random_seed_model,
                                                                                                config = config,
                                                                                                verbosity = -1) for index in range(config['make_classification']['num_eval']))

    for i, synthetic_result in enumerate(evaluation_results_synthetic):
        if i == 0:
            model_dict_synthetic = synthetic_result[0]
            scores_dict_synthetic = synthetic_result[1]
            dataset_dict_synthetic = synthetic_result[2]
        else: 
            model_dict_synthetic = mergeDict(model_dict_synthetic, synthetic_result[0])
            scores_dict_synthetic = mergeDict(scores_dict_synthetic, synthetic_result[1])
            dataset_dict_synthetic = mergeDict(dataset_dict_synthetic, synthetic_result[2])    
    
    del synthetic_result, evaluation_results_synthetic
    
    
    metric_identifer = '_test'
    metrics = ['accuracy', 'f1']
    index = [i for i in range(config['make_classification']['num_eval'])]
    columns = flatten_list([[[approach + ' ' + metric + '_mean', approach + ' ' + metric + '_max', approach + ' ' + metric + '_std'] for metric in metrics] for approach in ['DHDT', 'sklearn']])


    results_DHDT = None
    results_sklearn = None
    for metric in metrics:
        scores_DHDT = [scores_dict_synthetic[i]['DHDT'][metric + metric_identifer] for i in range(config['make_classification']['num_eval'])]

        scores_sklearn = [scores_dict_synthetic[i]['sklearn'][metric + metric_identifer] for i in range(config['make_classification']['num_eval'])]

        scores_DHDT_mean = np.mean(scores_DHDT, axis=1) if config['computation']['trials'] > 1 else scores_DHDT
        scores_sklearn_mean = np.mean(scores_sklearn, axis=1) if config['computation']['trials'] > 1 else scores_sklearn

        scores_DHDT_max = np.max(scores_DHDT, axis=1) if config['computation']['trials'] > 1 else scores_DHDT
        scores_sklearn_max = np.max(scores_sklearn, axis=1) if config['computation']['trials'] > 1 else scores_sklearn

        scores_DHDT_std = np.std(scores_DHDT, axis=1) if config['computation']['trials'] > 1 else np.array([0.0] * config['computation']['trials'])
        scores_sklearn_std = np.std(scores_sklearn, axis=1) if config['computation']['trials'] > 1 else np.array([0.0] * config['computation']['trials'])

        results_DHDT_by_metric = np.vstack([scores_DHDT_mean, scores_DHDT_max, scores_DHDT_std])
        results_sklearn_by_metric = np.vstack([scores_sklearn_mean, scores_sklearn_max, scores_sklearn_std])

        if results_DHDT is None and results_sklearn is None:
            results_DHDT = results_DHDT_by_metric
            results_sklearn = results_sklearn_by_metric
        else:
            results_DHDT = np.vstack([results_DHDT, results_DHDT_by_metric])
            results_sklearn = np.vstack([results_sklearn, results_sklearn_by_metric])

    scores_dataframe_synthetic = pd.DataFrame(data=np.vstack([results_DHDT, results_sklearn]).T, index = index, columns = columns)    
    
    del model_dict_synthetic, scores_dict_synthetic, dataset_dict_synthetic
    
    display(scores_dataframe_synthetic)
    display(scores_dataframe_synthetic[scores_dataframe_synthetic.columns[1::3]])
    display(scores_dataframe_synthetic.describe())

[Parallel(n_jobs=50)]: Using backend LokyBackend with 50 concurrent workers.
[Parallel(n_jobs=50)]: Done   2 out of  50 | elapsed:  5.3min remaining: 126.2min
[Parallel(n_jobs=50)]: Done  19 out of  50 | elapsed:  6.0min remaining:  9.8min
[Parallel(n_jobs=50)]: Done  36 out of  50 | elapsed:  6.5min remaining:  2.5min
[Parallel(n_jobs=50)]: Done  50 out of  50 | elapsed:  7.2min finished


,DHDT accuracy_mean,DHDT accuracy_max,DHDT accuracy_std,DHDT f1_mean,DHDT f1_max,DHDT f1_std,sklearn accuracy_mean,sklearn accuracy_max,sklearn accuracy_std,sklearn f1_mean,sklearn f1_max,sklearn f1_std
0,0.5658,0.636,0.047977,0.476465,0.601179,0.182926,0.6314,0.693,0.0308,0.623860,0.702807,0.039473
1,0.5848,0.615,0.026828,0.522209,0.592251,0.037229,0.6486,0.683,0.0172,0.611868,0.688299,0.038215
2,0.6256,0.647,0.024589,0.568280,0.672535,0.084312,0.7676,0.775,0.0148,0.770586,0.777448,0.013724
3,0.6876,0.717,0.022132,0.675633,0.705515,0.022849,0.7960,0.799,0.0060,0.803321,0.809479,0.012316
4,0.6154,0.642,0.015187,0.660878,0.679335,0.016190,0.6856,0.788,0.0512,0.690024,0.778243,0.044110
5,0.5438,0.587,0.030262,0.596350,0.645588,0.050531,0.6522,0.660,0.0156,0.639626,0.653768,0.028284
6,0.6684,0.699,0.028303,0.625272,0.693795,0.059461,0.7252,0.731,0.0116,0.707323,0.725230,0.035813
7,0.5742,0.631,0.034822,0.519235,0.586451,0.067822,0.6048,0.732,0.0636,0.541394,0.733068,0.095837
8,0.6764,0.754,0.083591,0.623312,0.775439,0.181032,0.7674,0.790,0.0452,0.766593,0.798464,0.063743
9,0.6250,0.647,0.024519,0.585374,0.637931,0.039031,0.7032,0.723,0.0396,0.701030,0.714138,0.026216


,DHDT accuracy_max,DHDT f1_max,sklearn accuracy_max,sklearn f1_max
0,0.636,0.601179,0.693,0.702807
1,0.615,0.592251,0.683,0.688299
2,0.647,0.672535,0.775,0.777448
3,0.717,0.705515,0.799,0.809479
4,0.642,0.679335,0.788,0.778243
5,0.587,0.645588,0.660,0.653768
6,0.699,0.693795,0.731,0.725230
7,0.631,0.586451,0.732,0.733068
8,0.754,0.775439,0.790,0.798464
9,0.647,0.637931,0.723,0.714138


,DHDT accuracy_mean,DHDT accuracy_max,DHDT accuracy_std,DHDT f1_mean,DHDT f1_max,DHDT f1_std,sklearn accuracy_mean,sklearn accuracy_max,sklearn accuracy_std,sklearn f1_mean,sklearn f1_max,sklearn f1_std
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,0.616656,0.673960,0.040038,0.583341,0.677513,0.088355,0.700432,0.734580,0.026920,0.696457,0.736784,0.032262
std,0.044567,0.052833,0.015856,0.065130,0.051497,0.050749,0.061110,0.058665,0.020653,0.067457,0.059481,0.028037
min,0.538600,0.584000,0.014711,0.439070,0.575556,0.016190,0.554600,0.589000,0.000800,0.541394,0.582106,0.000069
25%,0.585550,0.634500,0.026937,0.532264,0.641564,0.057053,0.651600,0.693250,0.010700,0.659153,0.705004,0.010640
50%,0.607300,0.669500,0.037026,0.590348,0.678248,0.071075,0.698500,0.733500,0.019000,0.707065,0.743031,0.027030
75%,0.656150,0.715000,0.053500,0.638117,0.714373,0.108539,0.756200,0.779750,0.044100,0.740445,0.782085,0.045441
max,0.711400,0.761000,0.083591,0.716268,0.779317,0.238641,0.798800,0.827000,0.068800,0.803321,0.833176,0.125842


## Real-World Eval

In [5]:
if True:

    identifier_list = [
                        'Adult',#: 32,
                        'Bank Marketing',#: 32,
                        'Loan Credit',#: 32,

                        'Credit Card',#: 23, 
                        'Car',#: 21,


                        'Absenteeism',#: 15,
                        'Loan House',#: 15,
                        'Cervical Cancer',#: 15,

                        'Heart Disease',#: 13,           

                        'Titanic',#: 10,
                        'Medical Insurance',#: 10,
                        'Wisconsin Breast Cancer Original',#: 10,
                        'Wisconsin Diagnostic Breast Cancer',#: 10,
                        'Wisconsin Prognostic Breast Cancer',#: 10,
                        'Abalone',#: 10,

                        'Habermans Survival',#: 3, 
                      ]

    identifier_list = ['Habermans Survival']

    parallel_eval_real_world = Parallel(n_jobs=config['computation']['n_jobs'], verbose=3, backend='loky') #loky #sequential multiprocessing
    evaluation_results_real_world = parallel_eval_real_world(delayed(evaluate_real_world_parallel)(identifier_list=identifier_list, 
                                                                                                   random_seed_model=config['computation']['random_seed']+i,
                                                                                                   config = config,
                                                                                                   verbosity = -1) for i in range(config['computation']['trials']))


    for i, real_world_result in enumerate(evaluation_results_real_world):
        if i == 0:
            model_dict_real_world = real_world_result[0]
            scores_dict_real_world = real_world_result[1]
            dataset_dict_real_world = real_world_result[2]
        else: 
            model_dict_real_world = mergeDict(model_dict_real_world, real_world_result[0])
            scores_dict_real_world = mergeDict(scores_dict_real_world, real_world_result[1])
            dataset_dict_real_world = mergeDict(dataset_dict_real_world, real_world_result[2])    

    del real_world_result, evaluation_results_real_world

    metric_identifer = '_test'
    metrics = ['accuracy', 'f1']
    index = identifier_list
    columns = flatten_list([[[approach + ' ' + metric + '_mean', approach + ' ' + metric + '_max', approach + ' ' + metric + '_std'] for metric in metrics] for approach in ['DHDT', 'sklearn']])


    results_DHDT = None
    results_sklearn = None
    for metric in metrics:
        scores_DHDT = [scores_dict_real_world[identifier]['DHDT'][metric + metric_identifer] for identifier in identifier_list]

        scores_sklearn = [scores_dict_real_world[identifier]['sklearn'][metric + metric_identifer] for identifier in identifier_list]    

        scores_DHDT_mean = np.mean(scores_DHDT, axis=1) if config['computation']['trials'] > 1 else scores_DHDT
        scores_sklearn_mean = np.mean(scores_sklearn, axis=1) if config['computation']['trials'] > 1 else scores_sklearn

        scores_DHDT_max = np.max(scores_DHDT, axis=1) if config['computation']['trials'] > 1 else scores_DHDT
        scores_sklearn_max = np.max(scores_sklearn, axis=1) if config['computation']['trials'] > 1 else scores_sklearn

        scores_DHDT_std = np.std(scores_DHDT, axis=1) if config['computation']['trials'] > 1 else np.array([0.0] * config['computation']['trials'])
        scores_sklearn_std = np.std(scores_sklearn, axis=1) if config['computation']['trials'] > 1 else np.array([0.0] * config['computation']['trials'])

        results_DHDT_by_metric = np.vstack([scores_DHDT_mean, scores_DHDT_max, scores_DHDT_std])
        results_sklearn_by_metric = np.vstack([scores_sklearn_mean, scores_sklearn_max, scores_sklearn_std])

        if results_DHDT is None and results_sklearn is None:
            results_DHDT = results_DHDT_by_metric
            results_sklearn = results_sklearn_by_metric
        else:
            results_DHDT = np.vstack([results_DHDT, results_DHDT_by_metric])
            results_sklearn = np.vstack([results_sklearn, results_sklearn_by_metric])

    del model_dict_real_world, scores_dict_real_world, dataset_dict_real_world
            
    scores_dataframe_real_world = pd.DataFrame(data=np.vstack([results_DHDT, results_sklearn]).T, index = index, columns = columns)
    display(scores_dataframe_real_world)
    display(scores_dataframe_real_world[scores_dataframe_real_world.columns[1::3]])    




[Parallel(n_jobs=50)]: Using backend LokyBackend with 50 concurrent workers.
[Parallel(n_jobs=50)]: Done   2 out of   5 | elapsed:   26.8s remaining:   40.2s
[Parallel(n_jobs=50)]: Done   5 out of   5 | elapsed:   27.0s finished


,DHDT accuracy_mean,DHDT accuracy_max,DHDT accuracy_std,DHDT f1_mean,DHDT f1_max,DHDT f1_std,sklearn accuracy_mean,sklearn accuracy_max,sklearn accuracy_std,sklearn f1_mean,sklearn f1_max,sklearn f1_std
Habermans Survival,0.695082,0.704918,0.008031,0.82009,0.826923,0.005579,0.688525,0.688525,0.0,0.791209,0.791209,0.0


,DHDT accuracy_max,DHDT f1_max,sklearn accuracy_max,sklearn f1_max
Habermans Survival,0.704918,0.826923,0.688525,0.791209


In [6]:
if False:
    identifier = identifier_list[0]#"Absenteeism"
    plt.figure(figsize=(15,8))
    image = model_dict_real_world[identifier]['DHDT'].plot(normalizer_list=dataset_dict_real_world[identifier]['normalizer_list'])
    display(image)

    plt.figure(figsize=(15,8))
    plot_tree(model_dict_real_world[identifier]['sklearn'], fontsize=10) 
    plt.show()